## Train Script for Bengali AI models
Load in model classes and processing scripts. Run training and evaluation here

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Packages
import torch
import torchvision
from torch.utils.data.sampler import WeightedRandomSampler
from torch.utils.data import DataLoader
from torch.autograd import Variable


from model.modelBase import *
from model.wrapperModel import *
from utils.evalUtils import *
from ProcessAndAugment import *

In [3]:
# paths
datadir = "./data"
inputdir= datadir + "/raw"
outputdir= datadir + "/processed"

In [4]:
# Parameters
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size=32

n_grapheme = 168
n_vowel = 11
n_consonant = 7
n_total = n_grapheme + n_vowel + n_consonant
print('n_total', n_total)

n_total 186


In [5]:
# Model Selection

# core model
predictor = densenet(in_channels=1, out_dim=n_total).to(device)
print('predictor', type(predictor))

# select our wrapper class
classifier = BengaliClassifier(predictor).to(device)
print('classifier',type(classifier))

predictor <class 'model.modelBase.densenet'>
classifier <class 'model.wrapperModel.BengaliClassifier'>


In [6]:
# Model Parameters
epochs = 10
lr = .001 # TODO: starting with flat LR, but need to implement scheduler
bs = 64

optimizer = torch.optim.Adam(classifier.parameters(), lr=lr)


# ignoring scheduler for now until we have baseline
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.7, patience=5, min_lr=1e-10)


validate_every = 5 # TODO: validate every n batches or epochs
checkpoint_every = 5 # TODO: implement model checkpoints

## Prep Data
Utilizes our process and data augmentation script

In [7]:
# load train file and generate dataset
train = pd.read_csv(datadir+'/train.csv')
indices = [0] # just set to list of all indices when actually training
dataset, crop_rsz_img = genDataset(indices, inputdir, train = train) # generates the dataset class

print(dataset.get_example(0))


image_df_list 1
~~Loaded Images~~
~~Standardized Images~~
(array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([15,  9,  5], dtype=int64))


In [8]:
# our weights for the weighted random sampler for each epoch
consonant_weights = genWeightTensor("consonant_diacritic", train[:len(crop_rsz_img)])
root_weights = genWeightTensor("grapheme_root", train[:len(crop_rsz_img)])
vowel_weights = genWeightTensor("vowel_diacritic", train[:len(crop_rsz_img)])
grapheme_weights = genWeightTensor("grapheme", train[:len(crop_rsz_img)])

weights = {"consonant_diacritic": consonant_weights,
           "grapheme_root": root_weights,
           "vowel_diacritic": vowel_weights,
           "grapheme": grapheme_weights}

#weight_keys = list(weights.keys())
# can change the focus of the sampler like so
weight_keys = ['grapheme', 'grapheme_root', 'vowel_diacritic', 'consonant_diacritic', 'grapheme', 'grapheme_root']

## Training

In [ ]:
for i, wkey in zip(range(epochs), itertools.cycle(weight_keys)):
    print(i, wkey)
    
    # generate sampler and loader specific to epoch
    wgt_val = weights[wkey]
    sampler = WeightedRandomSampler(wgt_val, len(wgt_val))
    train_loader = DataLoader(dataset, batch_size=bs, sampler=sampler)
    num_batches = len(train_loader)
    
    # init
    predictor.train()
    classifier.train()
    
    for j, (images, labels) in enumerate(train_loader):
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)
        
        # reset
        optimizer.zero_grad()
        
        # run model - requires 4d float input
        loss, metrics, pred = classifier(images.unsqueeze(1).float(), labels)
        
        # compute loss and step
        loss.backward()
        
        optimizer.step()
        
    print(metrics)

0 consonant_diacritic


C:\Users\ABell\Anaconda3\lib\site-packages\torch\utils\data\sampler.py:115: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.weights = torch.tensor(weights, dtype=torch.double)


{'loss': 4.869107246398926, 'loss_grapheme': 2.893148422241211, 'loss_vowel': 0.9825469851493835, 'loss_consonant': 0.9934120178222656, 'acc_grapheme': tensor(0.4118, device='cuda:0'), 'acc_vowel': tensor(0.7941, device='cuda:0'), 'acc_consonant': tensor(0.7059, device='cuda:0')}
1 grapheme_root
{'loss': 5.132531642913818, 'loss_grapheme': 3.5412003993988037, 'loss_vowel': 0.9875807166099548, 'loss_consonant': 0.6037505865097046, 'acc_grapheme': tensor(0.2941, device='cuda:0'), 'acc_vowel': tensor(0.7647, device='cuda:0'), 'acc_consonant': tensor(0.9412, device='cuda:0')}
2 vowel_diacritic
